<html>
    <h1 style="color:blue;text-align: center;">Feature Engineering</h1>
    <h2 style="color:green;text-align: center;">Column Transformer</h2>
</html>

In [1]:
# 1. Import the requests library
import requests

URL = "https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv"
# 2. download the data behind the URL
response = requests.get(URL)

# 3. Open the response into a new file called instagram.ico
open("covid_toy.csv", "wb").write(response.content)

3296

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('covid_toy.csv')
df.sample()

,age,gender,fever,cough,city,has_covid
35,82,Female,102.0,Strong,Bangalore,No


In [5]:
df['gender'].value_counts()

Female    59
Male      41
Name: gender, dtype: int64

In [7]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [8]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [9]:
df['has_covid'].value_counts()

No     55
Yes    45
Name: has_covid, dtype: int64

**We can apply `OrdinalEncoder` to `cough` column and `OneHotEncoder` to `gender` and `city` columns.**
From `age` columns we will get 2 columns and from `city` 4 columns. From `caugh` we will get `1` column for `OrdinalEncoder`.
Total 7 extra columns.

In [10]:
df['city'].shape

(100,)

In [12]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

**There are null values in fevel so we can use SimpleImputer() for this columns.**

We can do all this task seperately but using **Column Transformer** we can do all of these together.


# Applying ColumnTransformer()

In [17]:
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [50]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')#reminder has two value , drop for dropping others cols and 
                          #pasthrough does not make any chnage to other columns

In [36]:
# dir(transformer)

In [39]:
X_trainf_after_transformation = transformer.fit_transform(X_train)

In [44]:
X_trainf_after_transformation.shape

(80, 7)

In [41]:
X_train

,age,gender,fever,cough,city
50,19,Male,101.0,Mild,Delhi
28,16,Male,104.0,Mild,Kolkata
51,11,Female,100.0,Strong,Kolkata
74,34,Female,104.0,Strong,Delhi
43,22,Female,99.0,Mild,Bangalore
...,...,...,...,...,...
49,44,Male,104.0,Mild,Mumbai
38,49,Female,101.0,Mild,Delhi
72,83,Female,101.0,Mild,Kolkata
16,69,Female,103.0,Mild,Kolkata


In [43]:
type(X_trainf_after_transformation)

numpy.ndarray

In [48]:
df_new = pd.DataFrame(X_trainf_after_transformation, columns = ['gender_1','gender_2','cough','city_1','city_2','city_3','city_4'])

In [49]:
df_new

,gender_1,gender_2,cough,city_1,city_2,city_3,city_4
0,101.0,0.0,1.0,1.0,0.0,0.0,19.0
1,104.0,0.0,1.0,0.0,1.0,0.0,16.0
2,100.0,1.0,0.0,0.0,1.0,0.0,11.0
3,104.0,1.0,0.0,1.0,0.0,0.0,34.0
4,99.0,0.0,0.0,0.0,0.0,0.0,22.0
...,...,...,...,...,...,...,...
75,104.0,0.0,1.0,0.0,0.0,1.0,44.0
76,101.0,0.0,0.0,1.0,0.0,0.0,49.0
77,101.0,0.0,0.0,0.0,1.0,0.0,83.0
78,103.0,0.0,0.0,0.0,1.0,0.0,69.0
